In [69]:
from stravalib.client import Client
from stravalib import model
import json
import pandas as pd
import numpy as np
import polyline
import gpxpy
import folium
import ast
from collections import defaultdict, Counter
from pymongo import MongoClient
from math import radians, cos, sin, asin, sqrt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import scale
from math import radians, cos, sin, asin, sqrt

In [3]:
df = pd.read_csv('run_data_11-27-2018.csv')

In [15]:
df = df.iloc[:,1::]

In [18]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    paraphrased from 
    https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [19]:
def find_distances(coordinate1, coordinate2):
    lat1, lon1 = coordinate1
    lat2, lon2 = coordinate2
    distance = haversine(lat1, lon1, lat2, lon2)
    return distance


In [20]:
def make_floats(tup):
    x, y = tup
    return (float(x), float(y))

In [21]:
def get_distances(df, start):
    '''Takes in a dataframe of activities and returns a dataframe with start and end latlng as tuples with floats.
    Also returns the distance away from a starting point input.'''
    df_starts = df[(~df['start_latlng'].isna()) & (~df['end_latlng'].isna())]
    df_starts['start_latlng'] = df_starts['start_latlng'].apply(lambda x: x.split(","))
    df_starts['start_latlng'] = df_starts['start_latlng'].apply(lambda x: tuple(x))
    df_starts['start_latlng'] = df_starts['start_latlng'].apply(lambda x: make_floats(x))
    df_starts['end_latlng'] = df_starts['end_latlng'].apply(lambda x: x.split(","))
    df_starts['end_latlng'] = df_starts['end_latlng'].apply(lambda x: tuple(x))
    df_starts['end_latlng'] = df_starts['end_latlng'].apply(lambda x: make_floats(x))
    df_starts['distance_away'] = df_starts['start_latlng'].apply(lambda x: find_distances(start, x))
    return df_starts
    

### Start Here With Your Location

In [400]:
start = (47.529832, -121.987695)

In [401]:
df_starts = get_distances(df, start)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

In [402]:
working_df = df_starts[df_starts['distance_away']<= 3]

In [28]:
def standardize_inputs(user_input, df):
    '''Standardize the user inputs for cosine similarity'''
    elevation = user_input[0]
    distance = user_input[1]
    std_elevation = (elevation - df['total_elevation_gain'].mean())/df['total_elevation_gain'].std()
    std_distance = (distance - df['miles_converted'].mean())/df['miles_converted'].std()
    return np.array([std_elevation,std_distance])

In [403]:
def recommend_runs(request, df, columns_to_check):
    '''Inputs are a list of user-specified elevation gain in meters and miles to run, 
    dataframe of activities, and the columns of the dataframe to check 
    for cosine similarity. Columns to check should be in standardized form.  
    Output is a dictionary of polyline maps for route recommendations.'''
    
    #requires sklearn.cosine_similarity
    
    df['elevation_std'] = scale(df['total_elevation_gain'])
    df['miles_std'] = scale(df['miles_converted'])
    similarity_df = df.loc[:, columns_to_check]
    user_input = standardize_inputs(request, df)
    user_input = user_input.reshape(1,len(columns_to_check))
    user_input_reshaped = user_input.reshape(1,-1)
    similarities = cosine_similarity(similarity_df, user_input_reshaped)
    sort_indices = np.argsort(similarities, axis = None)
    top_20 = sort_indices[-20:]
    recommend_indices = list(top_20[::-1]) #reverse the order
    recommendations = df.iloc[recommend_indices, :]
    return dict(recommendations['map'])

### Get a dataframe of Recommendations Here

In [404]:
recommend_dict = recommend_runs([100, 5], working_df , ['elevation_std', 'miles_std'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [405]:
def make_polyline_lst(recommend_dict):
    '''Take in a dictionary of map objects and return dictionary of polylines{index:polyline} and the indices
    for the polylines as a list.'''
    polylines = {}
    for k, v in recommend_dict.items():
        v = ast.literal_eval(v)
        if v['summary_polyline'] != None: #make sure the polyline list isn't empty
            polylines[k] = v['summary_polyline']
    indices = list(polylines.keys())
    return polylines, indices

In [406]:
polylines,indices = make_polyline_lst(recommend_dict)

In [407]:
#get coordinates for the polylines in the list
map_coordinates = []
for line in list(polylines.values()):
    coordinates = polyline.decode(line)
    map_coordinates.append(coordinates)

In [27]:
def find_centroids(coordinate_lst):
    centroids = []
    for l in coordinate_lst:
        lats = []
        longs = []
        for point in l:
            lats.append(point[0])
            longs.append(point[1])
        centroid = (round(np.mean(lats), 3), round(np.mean(longs), 3))
        centroids.append(centroid)
    return centroids

In [113]:
def make_comparison_df(coordinate_lst, df, indices):
    centroids = find_centroids(coordinate_lst)
    lats = []
    longs = []
    elevation_lst = []
    for c in centroids:
        lats.append(c[0])
        longs.append(c[1])
    for idx in indices: #get the elevation for the runs in the suggestion list.
        row = df.loc[idx] 
        elevation_lst.append(row['total_elevation_gain'])
    comparison_df = pd.DataFrame({'lats': lats, 'longs':longs, 'elevation':elevation_lst})
    return comparison_df

In [408]:
comparison_df = make_comparison_df(map_coordinates, working_df, indices)

In [355]:
def cosine_sim_threshold_idx(df):
    '''Takes in a dataframe of map lat centroids, map long centroids, and elevation gain and returns a list
       of indices that are below a similarity threshold for each map'''
    comparison_array = df.values
    comparison_array_std = (comparison_array - np.mean(comparison_array, axis=0)) / np.std(comparison_array, axis=0)
    #make comparisons with all the datapoints in the comparison array
    cosine_sim_arr = cosine_similarity(comparison_array_std)
    idx_dict = {}
    for idx1, sim in enumerate(cosine_sim_arr): 
        #goes through cosine similarity arrays for each index 
        #and returns those that are below a similarity threshold
        lst = []
        for idx, num in enumerate(sim):
            if abs(num) > 0.994:
                lst.append(idx)
        idx_dict[idx1] = tuple(lst)
    groups = [set(x) for x in set(idx_dict.values())]
    unique_groups = merge_groups(groups)
    return unique_groups, groups
    

In [356]:
groups_unique, groups = cosine_sim_threshold_idx(comparison_df)

In [357]:
groups

[{1, 9, 19},
 {15},
 {0},
 {2},
 {3},
 {4},
 {6, 8, 10, 11},
 {5},
 {16},
 {1, 19},
 {1, 9},
 {17},
 {12},
 {7},
 {18},
 {13},
 {14}]

In [330]:
check_unions = []
for group in groups:
    if len(group) > 1:
        check_unions.append(group)

In [332]:
unique_groups = []
for group1 in check_unions:
    for group2 in check_unions:
        if group1 != group2 and group1.intersection(group2):
            merged = group1.union(group2)
            unique_groups.append(merged)

SyntaxError: unexpected EOF while parsing (<ipython-input-332-cbceadb12871>, line 3)

In [361]:
from scipy.cluster import  hierarchy

In [409]:
comparison_array = comparison_df.values
comparison_array_std = (comparison_array - np.mean(comparison_array, axis=0)) / np.std(comparison_array, axis=0)
#make comparisons with all the datapoints in the comparison array
cosine_sim_arr = cosine_similarity(comparison_array_std)

In [410]:
threshold = 0.1
Z = hierarchy.linkage(cosine_sim_arr, 'average', metric="cosine")
C = hierarchy.fcluster(Z, threshold, criterion="distance")
ids = list(range(20))
cluster_groups = defaultdict(list)
for idx, grouping in enumerate(C):
    cluster_groups[grouping].append(idx)



In [411]:
indices_to_use = []
for value in cluster_groups.values():
    indices_to_use.append(value[0])

In [320]:
def merge_groups(groups):
    #need to fix the case where there are chains of intersections:  A in B and B in C
    merged_groups = []
    unique_groups = []
    for group1 in groups:
        for group2 in groups:
            if group1 != group2 and group1.intersection(group2):
                merged = group1.union(group2)
                merged_groups.append(merged)
            else:
                merged_groups.append(group1)
    for g in merged_groups:
        if g not in unique_groups:
            unique_groups.append(g)
    return unique_groups
    

In [354]:
def get_indices(groups):
    sort_groups = sorted(groups, key=len)
    sort_groups = sort_groups[::-1]
    indices_to_use = []
    for group in sort_groups:
        group = list(group)
        indices_to_use.append(group[0])
    return indices_to_use

In [348]:
indices_to_use = []
for group in sort_groups:
    group = list(group)
    indices_to_use.append(group[0])

In [412]:
indices_to_use

[0, 2, 3, 4, 5, 15, 18, 19]

In [413]:
unique_coordinates = [map_coordinates[i] for i in indices_to_use]

In [414]:
lat, long = map_coordinates[0][0]
m = folium.Map(location=[lat, long], zoom_start=12.2)

In [415]:
for idx, route in enumerate(unique_coordinates[0:5]):
    colors = ['blue','green','red','orange','purple']
    folium.PolyLine(
            route,
            weight=2,
            color=colors[idx]
        ).add_to(m)

In [416]:
m